# **Part 1**

---



# **0. Install Machine Learning libray : Pycaret**

In [ ]:
!pip install pycaret

# **1.1. Import Libraries**

In [ ]:
import pandas as pd
import pycaret
import numpy as np
from sklearn.preprocessing import LabelEncoder
import plotly.express as px
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error , r2_score , mean_absolute_percentage_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller, kpss, ccf
import seaborn as sns
import matplotlib.pyplot as plt


# **1.2. Download Data**

In [ ]:
##############################################################################
dataset = pd.read_csv(########################################################)

# **3. Primary Data analysis**

## **1.3. Detection of missing values**

In [ ]:
dataset["time"] = pd.to_datetime(dataset["time"])
print ('Detect Missing Values')
dataset.isna().sum()

## **1.4 Check data shape and rows (---- TO COMPLETE----)**

In [ ]:
##########################################################

## **1.5. Display the five first rows**

In [ ]:
##########################################################

# **Part 2:  Data Preprocessing**

---



## **2.1 Create the columns of : hour, weekday and month (----TO COMPLETE----)**

In [ ]:
dataset['hour']    = #############################
dataset['weekday'] = #############################
dataset['month']   = #############################

## **2.2 Convert Temperature from Kelvin to Celsius (------TO COMPLETE------)**

In [ ]:
cities = ['Barcelona', 'Bilbao', 'Madrid', 'Seville', 'Valencia']
for city in cities:
  dataset[f"temp_max_{city}"] = #############################
  dataset[f"temp_min_{city}"] = #############################
  dataset[f"temp_{city}"]     = #############################

## **2.3 Generate the temperature range "Temp_range" for each city(-----TO COMPLETE------)**

In [ ]:
cities = ['Barcelona', 'Bilbao', 'Madrid', 'Seville', 'Valencia']
for city in cities:
  temp_max = dataset[f"temp_max_{city}"]
  temp_min = dataset[f"temp_min_{city}"]
  dataset[f"temp_range_{city}"] = ###########################   # To complete

## **2.4 Define the time as index**

In [ ]:
dataset=dataset.set_index("time")

## **2.5. Check the new shape of your data (---TO COMPLETE ---)**


In [ ]:
#######################################################################

In [ ]:
#######################################################################

# **Part 3: Data Visualization**

---



## **3.1 Visulaize data distribution per column (parameter)**

In [ ]:
sns.displot(dataset["total load actual"], kind="kde")

## **3.2 Analyse the Temperature parameters of the 5 cities**

In [ ]:
dataset[[f"temp_{city}" for city in ['Barcelona', 'Bilbao', 'Madrid', 'Seville', 'Valencia']]].describe()

## **3.3 Visulaize Temperature evolution curves of the 5 cities**

In [ ]:
fig = px.line(dataset,
              x=dataset.index,
              y= [f"temp_{city}" for city in ['Barcelona', 'Bilbao', 'Madrid', 'Seville', 'Valencia']],
              template = 'plotly_dark')

fig.show()

## **3.4 Calculate the Mean Load per Day**

In [ ]:
df_viz = dataset.copy()
df_viz["MA24"] = df_viz["total load actual"].rolling(24).mean()

In [ ]:
fig = px.line(df_viz.loc['2016-01-01 00:00:00':'2019-01-01 23:59:59'],
              x=df_viz.loc['2016-01-01 00:00:00':'2019-01-01 23:59:59'].index,
              y=["total load actual","MA24"], template = 'plotly_dark')

fig.show()


## **3.5 Features Scaling**

In [ ]:
scaler = MinMaxScaler()
columns = [f"temp_{city}" for city in ['Barcelona', 'Bilbao', 'Madrid', 'Seville', 'Valencia']] \
                                + ["MA24","total load actual"]

scaled_features  = df_viz[columns].copy()

scaled_features[columns]=scaler.fit_transform(df_viz[columns])

In [ ]:
fig = px.line(scaled_features.loc['2016-01-01 00:00:00':'2019-01-01 23:59:59'],
              x=scaled_features.loc['2016-01-01 00:00:00':'2019-01-01 23:59:59'].index,
              y=[f"temp_{city}" for city in ['Barcelona']] \
                                + ["MA24"], template = 'plotly_dark')

fig.show()

## **3.6 Calculate the correlation matrix using Person Formula (--------TO COMPLETE-----)**

In [ ]:
#####################################################################
#####################################################################

## **3.7 Sort the parameters based on their correlation with the Total Load**

In [ ]:
df_viz.corr()['total load actual'].sort_values(ascending=False)

# **Part 4: Machine Learning Models (PyCaret)**

---



## **4.1 Data Split (train/test)  --- TO COMPLETE ---**

In [ ]:
train_size = #####################################        train : 80%           test : 20%
train = dataset.iloc[:train_size]
test = dataset.iloc[train_size:]
#########################################                 training data size
#########################################                 test data size

train datasize: 28051
test data size: 7013


## **4.2 Import Time series data and init setup with Pycaret**

In [ ]:
from pycaret.regression import *
s = setup(data = train, test_data =test,
          target="total load actual", fold = 5,transform_target = False, session_id = 123)

## **4.3 Launch comparaison between machine learning models for electricity demand prediction**

In [ ]:
best=compare_models()

In [ ]:
predictions = predict_model(best, data=test)

## **4.4 Display the result curve : real vs. predicted electricity demand**

In [ ]:
fig = px.line(predictions, x=predictions.index, y=["total load actual","prediction_label"], template = 'plotly_dark')
fig.show()

# **Part 5: Deep learning Models (Pytroch lightning)**

---



In [ ]:
!pip install pytorch_lightning torchinfo

In [ ]:
import torch
import torch.nn as nn
from torchinfo import summary
from torch.utils.data import DataLoader, Dataset
from pytorch_lightning import LightningModule, Trainer, seed_everything
from pytorch_lightning.callbacks import  ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, r2_score
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import matplotlib.pyplot as plt

seed_everything(42, workers=True)
# sets seeds for numpy, torch and pyt
# this is required to make the results reproducible
torch.use_deterministic_algorithms(True)
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"


## **5.1.Data Split and Preparation (train/validation/test). (-----TO COMPLETE----)**

In [ ]:
train_size = ###########################
valid_size = ###########################
test_size  = ###########################
window_length = ########################

## **5.2 Model evaluation functions**

In [ ]:
def evaluate_model_with_predict(trainer, model, dataloader, scaler_y):

    predictions = trainer.predict(model, dataloader)

    # Flatten predictions and true values
    pred_list = []
    true_list = []

    for (features, labels), pred_batch in zip(dataloader, predictions):
        # Process features, labels, and the corresponding predictions
        pred_list.append(pred_batch.cpu().numpy())  # Adjust key if model output differs
        true_list.append(labels.cpu().numpy())  # Adjust key if model stores true values

    preds = np.concatenate(pred_list, axis=0)
    ground_truth = np.concatenate(true_list, axis=0)

    # Inverse transform predictions and true values
    preds_inv = scaler_y.inverse_transform(preds.reshape(-1, 1)).squeeze()
    ground_truth_inv = scaler_y.inverse_transform(ground_truth.reshape(-1, 1)).squeeze()

    # Create a DataFrame for predictions and true values
    prediction_df = pd.DataFrame({"pred": preds_inv, "y_true": ground_truth_inv})
    plt.figure(figsize=(12, 6))

    # Plot ground truth and predictions
    plt.plot(prediction_df.index, prediction_df['y_true'], label='Ground Truth', linewidth=2,color="blue")
    plt.plot(prediction_df.index, prediction_df['pred'], label='Predictions', linewidth=2, linestyle='--',color="red")

    plt.title("Predictions vs Ground Truth", fontsize=16)
    plt.xlabel("Index", fontsize=12)
    plt.ylabel("Values", fontsize=12)
    plt.legend(fontsize=12)

    plt.grid(alpha=0.3)
    plt.show()
    return prediction_df


In [ ]:
def create_animated_plot(prediction_df, max_elements=100):

    # Limit the data to the first `max_elements` hours
    prediction_df = prediction_df.iloc[:max_elements]
    fig, ax = plt.subplots()
    real_line, = ax.plot([], [], lw=2, label="Real Data", color="blue")
    predicted_line, = ax.plot([], [], lw=2, linestyle="--", label="Predicted Data", color="red")
    ax.legend()
    ax.set_xlim(prediction_df.index.min(), prediction_df.index.max())
    ax.set_ylim(prediction_df[['y_true', 'pred']].min().min(), prediction_df[['y_true', 'pred']].max().max())

    def init():
        real_line.set_data([], [])
        predicted_line.set_data([], [])
        return real_line, predicted_line

    def update(frame):
        # Update real data
        x_real = prediction_df.index[:frame]
        y_real = prediction_df['y_true'][:frame]
        real_line.set_data(x_real, y_real)

        # Update predicted data
        x_pred = prediction_df.index[:frame]
        y_pred = prediction_df['pred'][:frame]
        predicted_line.set_data(x_pred, y_pred)

        return real_line, predicted_line

    animation = FuncAnimation(fig, update, frames=len(prediction_df), init_func=init, blit=True)

    return HTML(animation.to_jshtml())

## **5.3 Data Scaling and Normalisation Function**

In [ ]:
def normalize_dataset(X,y):
  scaler_X = MinMaxScaler(feature_range=(0, 1))
  scaler_y = MinMaxScaler(feature_range=(0, 1))
  scaler_X.fit(X[:train_size])
  scaler_y.fit(y[:train_size])
  X_norm = scaler_X.transform(X)
  y_norm = scaler_y.transform(y)
  dataset_norm = np.concatenate((X_norm, y_norm), axis=1)
  return scaler_y, dataset_norm

## **5.4 Normalize our dataset (X:dataset, Y: total load)**

In [ ]:
X = dataset[dataset.columns.drop('total load actual')].values
y = dataset['total load actual'].values

y = y.reshape(-1, 1)
scaler_y, dataset_norm=normalize_dataset(X,y)

## **5.5 Time series creation and supervision**

In [ ]:
def multivariate_data(dataset, target, start_index, end_index, history_size, step=1):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset)

    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])
        labels.append(target[i])
    return np.array(data), np.array(labels)

## **5.6 Time series creation and supervision : train, valid & test**

In [ ]:
X_train, y_train = multivariate_data(dataset_norm, dataset_norm[:, -1],
                                     0, train_size, window_length)
X_val, y_val = multivariate_data(dataset_norm, dataset_norm[:, -1],
                                 train_size, train_size+valid_size,window_length)
X_test, y_test = multivariate_data(dataset_norm, dataset_norm[:, -1],
                                   train_size+valid_size, len(dataset)-1,window_length)
len(X_train),len(X_val),len(X_test)

(24520, 5235, 5236)

## **5.7 creation of pytorch dataset**

In [ ]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        # essential for dataloaders to estimate the dataset length
        return len(self.data)

    def __getitem__(self, idx):
        # convert x and y to tensors
        x = torch.tensor(self.data[idx], dtype=torch.float32)
        y = torch.tensor(self.labels[idx], dtype=torch.float32)
        return x, y


## **5.8 create dataloaders (train/val/test) -- TO COMPLETE--**

In [ ]:
# Create DataLoaders
train_dataset = TimeSeriesDataset(X_train, y_train)
val_dataset = TimeSeriesDataset(X_val, y_val)
test_dataset = TimeSeriesDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader =   ########################################################
test_loader =  ########################################################

# **5.9 define custom metrics metrics**

In [ ]:
def custom_MAE(y_true, y_pred,scaler):
    y_true = y_true.detach().cpu().numpy().reshape(-1, 1)
    y_pred = y_pred.detach().cpu().numpy().reshape(-1, 1)
    y_true_unscaled = scaler.inverse_transform(y_true)
    y_pred_unscaled = scaler.inverse_transform(y_pred)
    return np.mean(np.abs(y_true_unscaled - y_pred_unscaled))


def custom_MAPE(y_true, y_pred,scaler,epsilon=1e-7):
    y_true = y_true.detach().cpu().numpy().reshape(-1, 1)
    y_pred = y_pred.detach().cpu().numpy().reshape(-1, 1)
    y_true_unscaled = scaler.inverse_transform(y_true)
    y_pred_unscaled = scaler.inverse_transform(y_pred)
    diff = np.abs(y_true_unscaled - y_pred_unscaled)
    denom = np.maximum(np.abs(y_true_unscaled), epsilon)
    return np.mean(diff / denom)

def custom_r2_score(y_true, y_pred,scaler):
      y_true = y_true.detach().cpu().numpy().reshape(-1, 1)
      y_pred = y_pred.detach().cpu().numpy().reshape(-1, 1)
      y_true_unscaled = scaler.inverse_transform(y_true)
      y_pred_unscaled = scaler.inverse_transform(y_pred)
      r2 = r2_score(y_true_unscaled, y_pred_unscaled)
      return r2

# **5.10 creation of pytorchlightning class `BaseModel`**

   

In [ ]:
class BaseModel(LightningModule):
    def __init__(self, model, loss_fn, scaler_y, lr=6e-3):
        super(BaseModel, self).__init__()
        self.model = model
        self.loss_fn = loss_fn
        self.lr = lr
        self.scaler_y = scaler_y

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x).squeeze()
        loss = self.loss_fn(y_pred, y)
        mae = custom_MAE(y, y_pred, self.scaler_y)
        mape = custom_MAPE(y, y_pred, self.scaler_y)
        r2 = custom_r2_score(y, y_pred, self.scaler_y)
        self.log_dict({"train_loss": loss, "train_mae": mae, "train_mape": mape, "train_r2": r2}, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x).squeeze()
        loss = self.loss_fn(y_pred, y)
        mae = custom_MAE(y, y_pred, self.scaler_y)
        mape = custom_MAPE(y, y_pred, self.scaler_y)
        r2 = custom_r2_score(y, y_pred, self.scaler_y)
        self.log_dict({"val_loss": loss, "val_mae": mae, "val_mape": mape, "val_r2": r2}, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x).squeeze()
        loss = self.loss_fn(y_pred, y)
        mae = custom_MAE(y, y_pred, self.scaler_y)
        mape = custom_MAPE(y, y_pred, self.scaler_y)
        r2 = custom_r2_score(y, y_pred, self.scaler_y)
        self.log_dict({"test_loss": loss, "test_mae": mae, "test_mape": mape, "test_r2": r2}, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, y = batch
        return self(x).squeeze()

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)


# **Part 6: Define, train and evaluate MLP Model**

## **6.1 Create MLP Model**

In [ ]:

# **5. Model Definitions**
class MLPModel(nn.Module):
    def __init__(self):
        super(MLPModel, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.LazyLinear(128)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, 128)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

In [ ]:
mlp_model = MLPModel()

summary(mlp_model.cuda(),(1,24,57))

Layer (type:depth-idx)                   Output Shape              Param #
MLPModel                                 [1, 1]                    --
├─Flatten: 1-1                           [1, 1368]                 --
├─Linear: 1-2                            [1, 128]                  175,232
├─ReLU: 1-3                              [1, 128]                  --
├─Dropout: 1-4                           [1, 128]                  --
├─Linear: 1-5                            [1, 128]                  16,512
├─ReLU: 1-6                              [1, 128]                  --
├─Dropout: 1-7                           [1, 128]                  --
├─Linear: 1-8                            [1, 1]                    129
Total params: 191,873
Trainable params: 191,873
Non-trainable params: 0
Total mult-adds (M): 0.19
Input size (MB): 0.01
Forward/backward pass size (MB): 0.00
Params size (MB): 0.77
Estimated Total Size (MB): 0.78

## **6.2 Train MLP Model (---------TO COMPLETE--------)**

In [ ]:
# **7. Training Configuration**
loss_fn = nn.MSELoss()

model = BaseModel(mlp_model, loss_fn,scaler_y=scaler_y)

checkpoint_callback = ModelCheckpoint(".",filename="mlp",monitor="val_loss", save_top_k=1, mode="min")

trainer = Trainer(
    max_epochs=5,
    deterministic=True,
    callbacks=[checkpoint_callback]
)

# Train the Model**
trainer.fit(########)

## **6.3 Evaluate MLP Model**

In [ ]:
# test the model on test set
########################################################################

# **6.3.a. generate predictions using the function "evaluate_model_with_predict"**

In [ ]:
########################################################################

# **6.3.b. create animated plot using the function "create_animated_plot"**

In [ ]:
########################################################################

# **Part 7: Define, train and evaluate CNN Model**

---



## **7.1 Create CNN Model**

In [ ]:
import torch
import torch.nn as nn

class CNNModel(nn.Module):
    def __init__(self, input_shape):
        super(CNNModel, self).__init__()
        # Extract features and time steps from input_shape
        time_steps, features = input_shape
        self.conv1 = nn.Conv1d(in_channels=features, out_channels=64, kernel_size=6)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2)

        self.conv2 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size=2)

        self.flatten = nn.Flatten()
        self.dropout1 = nn.Dropout(0.3)
        self.fc1 = nn.LazyLinear(128)
        self.dropout2 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, 1)


    def forward(self, x):
        x = x.permute(0, 2, 1)  # Change to (batch_size, features, time_steps)
        x = torch.relu(self.conv1(x))
        x = self.pool1(x)
        x = torch.relu(self.conv2(x))
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.dropout1(x)
        x = self.fc1(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        return x


In [ ]:
input_shape = X_train.shape[-2:]
cnn_model = CNNModel(input_shape)

summary(cnn_model,(1,24,57))

Layer (type:depth-idx)                   Output Shape              Param #
CNNModel                                 [1, 1]                    --
├─Conv1d: 1-1                            [1, 64, 19]               21,952
├─MaxPool1d: 1-2                         [1, 64, 9]                --
├─Conv1d: 1-3                            [1, 64, 7]                12,352
├─MaxPool1d: 1-4                         [1, 64, 3]                --
├─Flatten: 1-5                           [1, 192]                  --
├─Dropout: 1-6                           [1, 192]                  --
├─Linear: 1-7                            [1, 128]                  24,704
├─Dropout: 1-8                           [1, 128]                  --
├─Linear: 1-9                            [1, 1]                    129
Total params: 59,137
Trainable params: 59,137
Non-trainable params: 0
Total mult-adds (M): 0.53
Input size (MB): 0.01
Forward/backward pass size (MB): 0.01
Params size (MB): 0.24
Estimated Total Size (MB): 0.26

## **7.2 train CNN Model**

In [ ]:
#######################################################################
#######################################################################
#######################################################################
#######################################################################

## **7.3 Evaluate CNN Model**

In [ ]:
# test the model on test set
######################################################################

In [ ]:
# generate predictions
######################################################################

In [ ]:
#create animated plot
######################################################################

# **Part 8: Define, train and evaluate RNN Model**

---



## **8.1 Create RNN Model**

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_shape):
        super(RNNModel, self).__init__()

        # Extract features and time steps from input_shape
        time_steps, features = input_shape

        self.rnn = nn.RNN(
            input_size=features,
            hidden_size=256,
            batch_first=True,
            nonlinearity='tanh'  # SimpleRNN uses tanh activation by default
        )
        self.fc1 = nn.Linear(256, 64)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        # Input shape: (batch_size, time_steps, features)
        # Output of RNN: (batch_size, time_steps, hidden_size) or (batch_size, hidden_size) if return_sequences=False
        _, h_n = self.rnn(x)  # h_n is the final hidden state of shape (1, batch_size, hidden_size)
        h_n = h_n.squeeze(0)  # Remove the first dimension since return_sequences=False
        x = self.fc1(h_n)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
# Check the RNN model architecture
#####################################################################
#####################################################################
#####################################################################

Layer (type:depth-idx)                   Output Shape              Param #
RNNModel                                 [1, 1]                    --
├─RNN: 1-1                               [1, 24, 256]              80,640
├─Linear: 1-2                            [1, 64]                   16,448
├─ReLU: 1-3                              [1, 64]                   --
├─Dropout: 1-4                           [1, 64]                   --
├─Linear: 1-5                            [1, 1]                    65
Total params: 97,153
Trainable params: 97,153
Non-trainable params: 0
Total mult-adds (M): 1.95
Input size (MB): 0.01
Forward/backward pass size (MB): 0.05
Params size (MB): 0.39
Estimated Total Size (MB): 0.44

## **8.2 Train RNN Model**

In [ ]:
#####################################################################
#####################################################################
#####################################################################

## **9.3 Evaluate RNN Model**

In [ ]:
# test the model on test set
######################################################################

In [ ]:
# generate predictions
######################################################################

In [ ]:
#create animated plot
######################################################################

# **Part 9: Define, train and evaluate LSTM Model**

---



## **9.1 Create LSTM Model**

In [ ]:
class LSTMModel(nn.Module):
    #########################################
        #########################################
        #########################################
        #########################################
        #########################################
        #########################################
        #########################################
        #########################################
        #########################################
        #########################################
        #########################################
        #########################################
        #########################################
        #########################################
        #########################################
        #########################################
        #########################################
        #########################################
        #########################################
        #########################################
        #########################################

In [ ]:
# check the LSTM model architecture
#####################################################################
#####################################################################
#####################################################################

## **9.2  Train  LSTM Model**

In [ ]:
######################################################################
######################################################################
######################################################################

## **9.3 Evaluate LSTM Model**

In [ ]:
# test the model on test set
######################################################################

In [ ]:
# generate predictions
######################################################################

In [ ]:
# create animated plot
######################################################################

# **Optional Part 10: Propose and optimize your best model**

---



In [ ]:
######################################################################
######################################################################
######################################################################

# **Optional Part 11: Define, train and evaluate GRU Model**

---



## **11.1 Create GRU Model**

In [ ]:
######################################################################
######################################################################
######################################################################
######################################################################
######################################################################
######################################################################
######################################################################
######################################################################
######################################################################
######################################################################
######################################################################
######################################################################
######################################################################
######################################################################
######################################################################

In [ ]:
########

## **11.2 Train GRU Model**

In [ ]:
######################################################################
######################################################################
######################################################################

## **11.3 Evaluate GRU Model**

In [ ]:
# test the model on test set
######################################################################

In [ ]:
# generate predictions
######################################################################

In [ ]:
# create animated plot
######################################################################